In [1]:
import argparse
import cv2
import numpy as np
import torch
from torch import nn
from torchvision import models
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam import GradCAM, \
    HiResCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad, \
    GradCAMElementWise

In [5]:
# VGG16
# model = models.vgg16()
# model.classifier[6] = nn.Linear(4096, 4)
# model.load_state_dict(torch.load('models/VGG16.pt', map_location=torch.device('cpu')))
# model.eval()

# target_layers = [model.features[-1]]


#ResNet50
model = models.resnet50()
model.fc = nn.Linear(2048, 4)
model.load_state_dict(torch.load('models/ResNet50_final.pt', map_location=torch.device('cpu')))
model.eval()

target_layers = [model.layer4]

In [6]:
rgb_img = cv2.imread('images/tests/0.jpg', 1)[:, :, ::-1]
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

In [8]:
targets = None

cam_algorithm = ScoreCAM
with cam_algorithm(model=model,
                   target_layers=target_layers,
                   use_cuda=False) as cam:

    cam.batch_size = 32
    grayscale_cam = cam(input_tensor=input_tensor,
                        targets=targets,
                        aug_smooth=False,
                        eigen_smooth=False)

    # Here grayscale_cam has only one image in the batch
    grayscale_cam = grayscale_cam[0, :]

    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    # cam_image is RGB encoded whereas "cv2.imwrite" requires BGR encoding.
    cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)

gb_model = GuidedBackpropReLUModel(model=model, use_cuda=False)
gb = gb_model(input_tensor, target_category=None)

cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
cam_gb = deprocess_image(cam_mask * gb)
gb = deprocess_image(gb)

cv2.imwrite(f'temp/1_cam.jpg', cam_image)

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 2510069760 bytes. Error code 12 (Cannot allocate memory)

In [36]:
preds = torch.argmax(model(input_tensor))
print(preds.item())

111


In [ ]:
    HiResCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad, \
    GradCAMElementWise